In [6]:
# Import necessary libraries
import ee
import geemap
import os
import rasterio
import numpy as np
import matplotlib.pyplot as plt
import rasterio
from rasterio.transform import from_origin
from rasterio.enums import Resampling
from rasterio.warp import reproject
import geopandas as gpd

In [7]:
# Authenticate and initialize the Earth Engine API
ee.Authenticate()
ee.Initialize()

In [8]:
# Create a map
Map = geemap.Map()
Map.add_basemap('Hybrid')


In [9]:
def addFile(file_path):
    file_extension = os.path.splitext(file_path)[1].lower()
    if file_extension == '.json' or file_extension == '.geojson':
        ee_object = geemap.geojson_to_ee(file_path)
    elif file_extension == '.shp':
        ee_object = geemap.shp_to_ee(file_path)
    else:
        raise ValueError("Unsupported file format. Supported formats: GeoJSON (.json, .geojson) and shapefile (.shp)")
    return ee_object

In [33]:
# Load and add boundary file
path =r"D:\pythonCourse05-04-2024\Git_Hub\Sample_Data.shp"
roi = addFile(path)
region = roi.geometry()
Map.centerObject(roi, zoom=14)
Map.addLayer(roi.style(**{'color': 'FFFFFF', 'fillColor': '00000000'}), {}, 'Boundary')

# Define date range
startdate = '2015-01-01'
enddate = '2024-03-30'

# Change dataset to Landsat 8
dataset = (
    ee.ImageCollection('LANDSAT/LC08/C02/T1_L2')
    .filterDate(startdate, enddate)
    .filterBounds(roi)
)


In [34]:
# Visualization parameters
visualization = {
    'min': 0,
    'max': 3000,
    'bands': ['SR_B6', 'SR_B5', 'SR_B4'],  # Correct band names
}

landsat8 = dataset.median().clip(roi)


In [35]:
# Check available bands
bands = landsat8.bandNames().getInfo()
print(f"Available bands: {bands}")

Available bands: ['SR_B1', 'SR_B2', 'SR_B3', 'SR_B4', 'SR_B5', 'SR_B6', 'SR_B7', 'SR_QA_AEROSOL', 'ST_B10', 'ST_ATRAN', 'ST_CDIST', 'ST_DRAD', 'ST_EMIS', 'ST_EMSD', 'ST_QA', 'ST_TRAD', 'ST_URAD', 'QA_PIXEL', 'QA_RADSAT']


In [36]:
# Calculate NDVI
red_band = 'SR_B4'
nir_band = 'SR_B5'
if red_band in bands and nir_band in bands:
    red = landsat8.select(red_band)
    nir = landsat8.select(nir_band)
    ndvi = nir.subtract(red).divide(nir.add(red)).rename('NDVI')

    # Display NDVI on map
    Map.addLayer(ndvi, {'min': -1, 'max': 1, 'palette': ['red', 'yellow', 'green']}, 'NDVI', True)

    # Export NDVI
    ndvi_output_path = r"D:\pythonCourse05-04-2024\Git_Hub\NDVI.tif"
    geemap.ee_export_image(
        ndvi,
        
        filename=ndvi_output_path,
        scale=30,
        region=region,
        file_per_band=False
    )
    print(f"NDVI calculation and export completed. Saved to {ndvi_output_path}")
else:
    print(f"Bands {red_band} and {nir_band} are not available in the dataset.")


Generating URL ...
Please wait ...
Data downloaded to D:\pythonCourse05-04-2024\Git_Hub\NDVI.tif
NDVI calculation and export completed. Saved to D:\pythonCourse05-04-2024\Git_Hub\NDVI.tif


In [37]:
# Calculate VCI
def calculate_vci(ndvi, historical_min, historical_max):
    historical_min_img = ee.Image.constant(historical_min)
    historical_max_img = ee.Image.constant(historical_max)
    vci = ndvi.subtract(historical_min_img).divide(historical_max_img.subtract(historical_min_img)).multiply(100).rename('VCI')
    return vci

# Example historical NDVI min and max values
historical_min = ndvi.reduceRegion(reducer=ee.Reducer.min(), geometry=roi, scale=30).getInfo()['NDVI']
historical_max = ndvi.reduceRegion(reducer=ee.Reducer.max(), geometry=roi, scale=30).getInfo()['NDVI']

vci = calculate_vci(ndvi, historical_min, historical_max)
Map.addLayer(vci, {'min': 0, 'max': 100, 'palette': ['red', 'yellow', 'green']}, 'VCI', True)

# Export VCI
vci_output_path = r"D:\pythonCourse05-04-2024\Git_Hub\VCI.tif"
geemap.ee_export_image(
    vci,
    filename=vci_output_path,
    scale=30,
    region=region,
    file_per_band=False
)
print(f"VCI calculation completed and saved to {vci_output_path}")


Generating URL ...
Please wait ...
Data downloaded to D:\pythonCourse05-04-2024\Git_Hub\VCI.tif
VCI calculation completed and saved to D:\pythonCourse05-04-2024\Git_Hub\VCI.tif


In [38]:
#Calculate LST

# ML_B10 = 0.000334  # 3.34E-04 in decimal form
# AL_B10 = 0.1
def dn_to_radiance(image, band, ml, al):
    radiance = image.select(band).multiply(ml).add(al)
    return radiance.rename('Radiance')

#  Lλ = ML×Qcal+AL
# Lλ is the TOA spectral radiance (Watts/(m² * sr * µm))
# 𝑀 𝐿is the band-specific multiplicative rescaling factor (RADIANCE_MULT_BAND_x)
# 𝐴𝐿 is the band-specific additive rescaling factor (RADIANCE_ADD_BAND_x)
# Qcal is the quantized and calibrated standard product pixel values (DN)

# Function to convert Radiance to Temperature
def radiance_to_temperature(radiance, k1, k2):
    temperature = radiance.expression(
        'K2 / log((K1 / Radiance) + 1)', {
            'Radiance': radiance.select('Radiance'),
            'K1': k1,
            'K2': k2
        }).rename('Temperature')
    return temperature


# Function to convert Kelvin to Celsius
def kelvin_to_celsius(temperature):
    celsius = temperature.subtract(273.15).rename('Celsius_Temperature')
    return celsius


# Function to calculate Emissivity
def calculate_emissivity(ndvi):
    ndvi_min = ee.Number(ndvi.reduceRegion(reducer=ee.Reducer.min(), geometry=roi, scale=30).get('NDVI'))
    ndvi_max = ee.Number(ndvi.reduceRegion(reducer=ee.Reducer.max(), geometry=roi, scale=30).get('NDVI'))

    pv = ndvi.subtract(ndvi_min).divide(ndvi_max.subtract(ndvi_min)).pow(2).rename('PV')
    emissivity = pv.multiply(0.004).add(0.986).rename('Emissivity')
    return emissivity
# Function to calculate LST
def calculate_lst(bt, emissivity):
    w = 0.00115
    p = 14380

    lst = bt.expression(
        '(BT / (1 + (w * BT / p) * log(e)))', {
            'BT': bt.select('Celsius_Temperature'),
            'w': w,
            'p': p,
            'e': emissivity.select('Emissivity')
        }).rename('LST')
    return lst

# Landsat 8 thermal band (Band 10)
lst_band = 'ST_B10'
if lst_band in bands:
    # Conversion parameters from metadata
    ml = 0.000334 
    al = 0.10
    k1 = 774.885
    k2 = 1321.08

    # Convert DN to Radiance
    radiance = dn_to_radiance(landsat8, lst_band, ml, al)

    # Convert Radiance to Temperature in Kelvin
    temperature = radiance_to_temperature(radiance, k1, k2)

    # Convert Temperature to Celsius
    celsius_temperature = kelvin_to_celsius(temperature)

# #      Calculate NDVI
#     ndvi = calculate_ndvi(ndvi)

    # Calculate Emissivity
    emissivity = calculate_emissivity(ndvi)

    # Calculate LST
    lst = calculate_lst(celsius_temperature, emissivity)

    
    Map.addLayer(lst, {'min': 0, 'max': 100, 'palette': ['blue', 'green', 'red']}, 'LST', True)

    # Export LST
    lst_output_path = r"D:\pythonCourse05-04-2024\Git_Hub\LST.tif"
    print(f"Starting lst export to {lst_output_path}")
    geemap.ee_export_image(
        lst,
        filename=lst_output_path,
        scale=30,
        region=region,
        file_per_band=False
    )
    print(f"LST calculation completed and saved to {lst_output_path}")
else:
    print(f"Band {lst_band} is not available in the dataset.")  

Starting lst export to D:\pythonCourse05-04-2024\Git_Hub\LST.tif
Generating URL ...
Please wait ...
Data downloaded to D:\pythonCourse05-04-2024\Git_Hub\LST.tif
LST calculation completed and saved to D:\pythonCourse05-04-2024\Git_Hub\LST.tif


In [39]:
# Calculate TCI
def calculate_tci(lst, historical_min, historical_max):
    historical_min_img = ee.Image.constant(historical_min)
    historical_max_img = ee.Image.constant(historical_max)
    tci = historical_max_img.subtract(lst).divide(historical_max_img.subtract(historical_min_img)).multiply(100).rename('TCI')
    return tci

    # Example historical LST min and max values
historical_min_lst = lst.reduceRegion(reducer=ee.Reducer.min(), geometry=roi, scale=30).getInfo()['LST']
historical_max_lst = lst.reduceRegion(reducer=ee.Reducer.max(), geometry=roi, scale=30).getInfo()['LST']

tci = calculate_tci(lst, historical_min_lst, historical_max_lst)
Map.addLayer(tci, {'min': 0, 'max': 100, 'palette': ['blue', 'yellow', 'red']}, 'TCI', True)

    # Export TCI
tci_output_path ="D:\pythonCourse05-04-2024\Git_Hub\TCI.tif"
print(f"Starting TCI export to {tci_output_path}")
geemap.ee_export_image(
        tci,
        filename=tci_output_path,
        scale=30,
        region=region,
        file_per_band=False
    )
print(f"TCI calculation completed and saved to {tci_output_path}")
# else:
#     print(f"Band {lst_band} is not available in the dataset.")


Starting TCI export to D:\pythonCourse05-04-2024\Git_Hub\TCI.tif
Generating URL ...
Please wait ...
Data downloaded to D:\pythonCourse05-04-2024\Git_Hub\TCI.tif
TCI calculation completed and saved to D:\pythonCourse05-04-2024\Git_Hub\TCI.tif


In [40]:
# Calculate VHI
def calculate_vhi(vci, tci, alpha=0.5, beta=0.5):
    vhi = vci.multiply(alpha).add(tci.multiply(beta)).rename('VHI')
    return vhi

if 'TCI' in tci.bandNames().getInfo() and 'VCI' in vci.bandNames().getInfo():
    vhi = calculate_vhi(vci, tci)
    Map.addLayer(vhi, {'min': 0, 'max': 100, 'palette': ['brown', 'yellow', 'green']}, 'VHI', True)

    # Export VHI
    vhi_output_path = r"D:\pythonCourse05-04-2024\Git_Hub\VHI.tif"
    print(f"Starting VHI export to {vhi_output_path}")
    geemap.ee_export_image(
        vhi,
        filename=vhi_output_path,
        scale=30,
        region=region,
        file_per_band=False
    )
    print(f"VHI calculation completed and saved to {vhi_output_path}")
else:
    print("VCI or TCI bands are not available in the dataset.")
    


Starting VHI export to D:\pythonCourse05-04-2024\Git_Hub\VHI.tif
Generating URL ...
Please wait ...
Data downloaded to D:\pythonCourse05-04-2024\Git_Hub\VHI.tif
VHI calculation completed and saved to D:\pythonCourse05-04-2024\Git_Hub\VHI.tif


In [41]:
# Function to calculate DSI
def calculate_dsi(ndvi, lst, ndvi_min, ndvi_max, lst_min, lst_max):
    ndvi_norm = ndvi.subtract(ndvi_min).divide(ndvi_max.subtract(ndvi_min))
    lst_norm = lst_max.subtract(lst).divide(lst_max.subtract(lst_min))
    dsi = ndvi_norm.multiply(lst_norm).rename('DSI')
    return dsi

# Calculate historical min and max values for NDVI and LST
ndvi_min = ndvi.reduceRegion(reducer=ee.Reducer.min(), geometry=roi, scale=30).getInfo()['NDVI']
ndvi_max = ndvi.reduceRegion(reducer=ee.Reducer.max(), geometry=roi, scale=30).getInfo()['NDVI']
lst_min = lst.reduceRegion(reducer=ee.Reducer.min(), geometry=roi, scale=30).getInfo()['LST']
lst_max = lst.reduceRegion(reducer=ee.Reducer.max(), geometry=roi, scale=30).getInfo()['LST']

# Calculate DSI
dsi = calculate_dsi(ndvi, lst, ee.Image.constant(ndvi_min), ee.Image.constant(ndvi_max), 
                    ee.Image.constant(lst_min), ee.Image.constant(lst_max))

# Add DSI to the map
Map.addLayer(dsi, {'min': 0, 'max': 1, 'palette': ['blue', 'green', 'yellow', 'red']}, 'DSI', True)

# Export DSI
dsi_output_path = r"D:\pythonCourse05-04-2024\Git_Hub\DSI.tif" 
print(f"Starting DSI export to {dsi_output_path}")
geemap.ee_export_image(
    dsi,
    filename=dsi_output_path,
    scale=30,
    region=region,
    file_per_band=False
)
print(f"DSI calculation completed and saved to {dsi_output_path}")

Starting DSI export to D:\pythonCourse05-04-2024\Git_Hub\DSI.tif
Generating URL ...
Please wait ...
Data downloaded to D:\pythonCourse05-04-2024\Git_Hub\DSI.tif
DSI calculation completed and saved to D:\pythonCourse05-04-2024\Git_Hub\DSI.tif


In [42]:
Map

Map(center=[-3.140319616036173, 130.38339122982435], controls=(WidgetControl(options=['position', 'transparent…